In [1]:
#this code is used to test the accuracy of the optiaml parameters for all 4 models
import numpy as np
from datetime import datetime
from dataloader import *
from knnmodel import *
from sklearn.model_selection import ShuffleSplit
from svmmodel import *
from datetime import datetime

In [2]:
#load data
train_mx, train_my, test_mx, test_my = loadmnist()
train_cx, train_cy, test_cx, test_cy = loadcatech()

In [3]:
#KNN for both datasets
def RunKnn(k,p,train_x,train_y,test_x,test_y):
## running knn
    #result=np.zeros((2,3))
    a = datetime.now()
    accuracy = 0
    test_num = test_x.shape[0]
    matchCount = 0
    for i in range(test_num):
        predict = kNNClassify(test_x[i], train_x, train_y, k,p)
        if predict == test_y[i]:
            matchCount += 1
    accuracy = (float(matchCount) / test_num)
    b = datetime.now()
    #result[0,p-1] = accuracy
    result = (b-a).seconds
    print("k=: ",k,"p=: ",p,accuracy,result)
    return accuracy, result

In [3]:
#svm for mnist
def Runsvmmnist(k,const,train_x,train_y,test_x,test_y):
    a = datetime.now()
    name=['linear','gaussian','poly_2']
    if k ==0:
        kern = linear_kernel
    if k ==1:
        kern = gaussian_kernel
    if k ==2:
        kern = polynomial_kernel
    accuracy = 0
# create, train and do prediction with multiple svms for each categories
    final_result = np.zeros(test_y.shape[0])
    count=0
    correct=0
    list = [SVM(C=const,kernel=kern) for i in range(0,10)]
    for i in range(0,10):
            #split dataset into 10 small folders for each category
        traindata  = train_x[count:(i+1)*6000,:]
        trainlabel = train_y[count:(i+1)*6000] - i + 1

        #for catech dataset, don't need to split the dataset, just use the whole traindata 10 times, so the code should be
        #traindata = train_x
        #trainlabel = train_y - i + 1

        trainlabel=np.array(np.where(trainlabel == 1, trainlabel, -1))
        count=count+6000
        list[i].fit(traindata, trainlabel)
        predictions = list[i].predict(test_x)
        final_result=np.array(np.where(predictions !=1, final_result, i))
    correct = np.sum(final_result == test_y)
    accuracy = float(correct)/(test_y.shape[0])
    b = datetime.now()
#calculate the accuracy of our prediction
    result = (b-a).seconds
    print("k=",name[k],"c=",const,accuracy,result)
    return accuracy, result

In [15]:
#svm for caltech dataset
def Runsvmcaltech(k,const,train_x,train_y,test_x,test_y):
    a = datetime.now()
    name=['linear','gaussian','poly_2']
    if k ==0:
        kern = linear_kernel
    if k ==1:
        kern = gaussian_kernel
    if k ==2:
        kern = polynomial_kernel
    rs = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
    accuracy = 0
# create, train and do prediction with multiple svms for each categories
    final_result = np.zeros(test_y.shape[0])
    correct=0
    list = [SVM(C=const,kernel=kern) for i in range(0,10)]
    for i in range(0,10):
            #split dataset into 10 small folders for each category
            #traindata  = train_x[k:(i+1)*6000,:]
            #trainlabel = train_y[k:(i+1)*6000] - i + 1
        traindata = train_x
        trainlabel = train_y -i +1

        #for catech dataset, don't need to split the dataset, just use the whole traindata 10 times, so the code should be
        #traindata = train_x
        #trainlabel = train_y - i + 1

        trainlabel=np.array(np.where(trainlabel == 1, trainlabel, -1))
            #k=k+6000
        list[i].fit(traindata, trainlabel)
        predictions = list[i].predict(test_x)
        final_result=np.array(np.where(predictions !=1, final_result, i))
    correct = np.sum(final_result == test_y)
    accuracy = float(correct)/(test_y.shape[0])
    b = datetime.now()
#calculate the accuracy of our prediction
    result = (b-a).seconds
    print("k=",name[k],"c=",const,accuracy,result)
    return accuracy, result

# MNIST_KNN--k=1,p=4

In [17]:
#mnist_knn(k=1,p=4)
accuracy_mk, result_mk = RunKnn(1,4,train_mx,train_my,test_mx,test_my)

k=:  1 p=:  4 0.7437 9585


In [8]:
#print the result
print("MNIST_KNN--k=1,p=4")
print("accuracy:",accuracy_mk)
print("time",result_mk)

MNIST_KNN--k=1,p=4
accuracy: 0.7437
time 9585


# CALTECH_KNN--K=3,P=6

In [9]:
#caltech_knn(k3,p=6)
X = np.concatenate((train_cx,test_cx), axis=0)
#dimension reduction by PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=1000)
pca.fit(X)
train_ckx = pca.transform(train_cx)
test_ckx = pca.transform(test_cx)
accuracy_ck, result_ck = RunKnn(3,6,train_ckx,train_cy,test_ckx,test_cy)

k=:  3 p=:  6 0.5077399380804953 4


In [10]:
#print the result
print("CALTECH_KNN--K=3,P=6")
print("accuracy:",accuracy_ck)
print("time",result_ck)

CALTECH_KNN--K=3,P=6
accuracy: 0.5077399380804953
time 4


# MNIST_SVM--K=poly_2,C=0.03

In [4]:
#reset the type of data to double for the calculation requirement
train_msx=train_mx.astype(np.double)
train_msy=train_my.astype(np.double)
test_msx=test_mx.astype(np.double)
test_msy=test_my.astype(np.double)

#apply pac for dimensionn reduction
#for catech data set, set pca to 25 components
from sklearn.decomposition import PCA
X = np.concatenate((train_msx,test_msx), axis=0)
pca = PCA(n_components=25)
pca.fit(X)
train_msx = pca.transform(train_msx)
test_msx  = pca.transform(test_msx)

#run the svm_mnist
accuracy_ms, result_ms = Runsvmmnist(2,0.03,train_msx,train_msy,test_msx,test_msy)


     pcost       dcost       gap    pres   dres
 0: -9.6683e+01 -3.5232e+02  5e+04  9e+01  7e-12
 1: -5.7275e+00 -3.4211e+02  1e+03  2e+00  6e-12
 2: -3.5894e+00 -1.3628e+02  2e+02  2e-01  8e-13
 3: -2.0047e+00 -5.8446e+01  8e+01  8e-02  4e-13
 4: -9.3541e-01 -3.1046e+01  4e+01  4e-02  2e-13
 5: -6.2405e-01 -2.4896e+01  4e+01  3e-02  1e-13
 6: -3.4438e-01 -1.8016e+01  3e+01  2e-02  9e-14
 7: -1.2810e-01 -1.0166e+01  1e+01  8e-03  5e-14
 8: -7.4603e-02 -5.1294e+00  7e+00  4e-03  3e-14
 9:  4.9711e-03 -4.4448e+00  6e+00  3e-03  2e-14
10:  7.5615e-03 -1.6774e+00  2e+00  9e-04  1e-14
11: -4.0225e-03 -4.1164e-01  4e-01  5e-05  1e-14
12: -3.2891e-02 -2.7025e-01  2e-01  2e-05  1e-14
13: -4.3941e-02 -2.3277e-01  2e-01  5e-06  1e-14
14: -6.6823e-02 -1.6278e-01  1e-01  2e-06  1e-14
15: -7.9959e-02 -1.2937e-01  5e-02  9e-08  1e-14
16: -8.6578e-02 -1.1576e-01  3e-02  2e-08  1e-14
17: -9.3856e-02 -1.0397e-01  1e-02  4e-09  1e-14
18: -9.6954e-02 -9.9100e-02  2e-03  3e-10  1e-14
19: -9.7791e-02 -9.80

 5: -2.6215e+00 -3.0351e+01  4e+01  3e-02  1e-13
 6: -1.8834e+00 -2.0603e+01  3e+01  2e-02  1e-13
 7: -1.3166e+00 -1.2989e+01  2e+01  1e-02  8e-14
 8: -9.2968e-01 -7.9550e+00  1e+01  5e-03  7e-14
 9: -6.8026e-01 -5.8490e+00  7e+00  3e-03  6e-14
10: -7.1092e-01 -3.0371e+00  3e+00  1e-03  6e-14
11: -6.9271e-01 -1.7183e+00  1e+00  2e-04  6e-14
12: -8.0479e-01 -1.1990e+00  4e-01  3e-05  6e-14
13: -8.7659e-01 -1.0224e+00  1e-01  2e-16  7e-14
14: -9.1529e-01 -9.6891e-01  5e-02  2e-16  6e-14
15: -9.3276e-01 -9.4481e-01  1e-02  4e-16  7e-14
16: -9.3809e-01 -9.3882e-01  7e-04  3e-16  7e-14
17: -9.3843e-01 -9.3844e-01  1e-05  3e-16  7e-14
18: -9.3843e-01 -9.3843e-01  2e-07  5e-16  8e-14
Optimal solution found.
224 support vectors out of 6000 points
     pcost       dcost       gap    pres   dres
 0: -2.4945e+02 -3.6571e+02  5e+04  9e+01  7e-12
 1: -1.5967e+01 -3.5582e+02  1e+03  2e+00  6e-12
 2: -1.0036e+01 -1.7863e+02  3e+02  3e-01  9e-13
 3: -6.7631e+00 -8.1882e+01  1e+02  1e-01  4e-13
 4: -4.

In [6]:
#print the result
print("MNIST_SVM--K=poly_2,C=0.03")
print("accuracy:",accuracy_ms)
print("time",result_ms)

MNIST_SVM--K=poly_2,C=0.03
accuracy: 0.9121
time 3252


# CALTECH_SVM--K=linear_kernel,C=0.01

In [16]:
#reset the type of data to double for the calculation requirement
train_csx=train_cx.astype(np.double)
train_csy=train_cy.astype(np.double)
test_csx=test_cx.astype(np.double)
test_csy=test_cy.astype(np.double)

#apply pac for dimensionn reduction
#for catech data set, set pca to 25 components
from sklearn.decomposition import PCA
X = np.concatenate((train_csx,test_csx), axis=0)
pca = PCA(n_components=25)
pca.fit(X)
train_csx = pca.transform(train_csx)
test_csx  = pca.transform(test_csx)

#run the svm_mnist
accuracy_cs, result_cs = Runsvmcaltech(0,0.01,train_csx,train_csy,test_csx,test_csy)


     pcost       dcost       gap    pres   dres
 0: -3.9732e+01 -6.3277e+00  2e+03  5e+01  3e-13
 1: -1.1254e+00 -6.2730e+00  4e+01  7e-01  2e-13
 2: -6.2539e-01 -4.2024e+00  7e+00  1e-01  3e-14
 3: -4.8877e-01 -2.1634e+00  3e+00  3e-02  1e-14
 4: -4.0337e-01 -1.0106e+00  9e-01  1e-02  5e-15
 5: -3.8570e-01 -6.0520e-01  3e-01  3e-03  3e-15
 6: -4.0274e-01 -4.5821e-01  7e-02  6e-04  2e-15
 7: -4.1143e-01 -4.2195e-01  1e-02  7e-05  2e-15
 8: -4.1417e-01 -4.1516e-01  1e-03  3e-06  2e-15
 9: -4.1453e-01 -4.1460e-01  8e-05  2e-07  2e-15
10: -4.1456e-01 -4.1456e-01  4e-06  1e-08  2e-15
11: -4.1456e-01 -4.1456e-01  1e-07  2e-10  2e-15
Optimal solution found.
61 support vectors out of 323 points
     pcost       dcost       gap    pres   dres
 0: -3.9894e+01 -6.4735e+00  2e+03  5e+01  3e-13
 1: -1.0956e+00 -6.4190e+00  4e+01  6e-01  3e-13
 2: -6.2329e-01 -4.2063e+00  7e+00  9e-02  4e-14
 3: -4.7678e-01 -1.7915e+00  2e+00  2e-02  9e-15
 4: -4.4143e-01 -8.6533e-01  5e-01  5e-03  3e-15
 5: -4.661

In [12]:
#print the result
print("CALTECH_SVM--K=linear_kernel,C=0.01")
print("accuracy:",accuracy_cs)
print("time",result_cs)

CALTECH_SVM--K=linear_kernel,C=0.01
accuracy: 0.5851393188854489
time 2
